In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
from scipy.signal import stft, istft

In [2]:
import tensorflow as tf

In [3]:
import os

sample_rate = 44100


def get_vocals_mask(Zxx, threshold=1):
    binary_mask = (np.abs(Zxx) > threshold).astype(float)
    return binary_mask

def get_ideal_binary_mask(Zxx_vocals, Zxx_mix, threshold=1):
    binary_mask = (np.abs(Zxx_vocals) > threshold * np.abs(Zxx_mix)).astype(float)
    return binary_mask

def get_inverse_mask(binary_mask):
    return (binary_mask == 0).astype(float)

def get_track_list():
    return os.listdir('data/musdb/musdb18hq/train')

def read_track(filename):
    directory = f'data/musdb/musdb18hq/train/{filename}'
    sample_rate, vocals = wav.read(f'{directory}/vocals.wav')
    sample_rate, mix = wav.read(f'{directory}/mixture.wav')

    return sample_rate, mix, vocals

def transform_track(signal, sample_rate, nperseg=2048, noverlap=2048 // 2):
    frequencies, times, Zxx_l = stft(signal[:, 0], fs=sample_rate, nperseg=nperseg, noverlap=noverlap, window='hamming')
    frequencies, times, Zxx_r = stft(signal[:, 1], fs=sample_rate, nperseg=nperseg, noverlap=noverlap, window='hamming')

    return frequencies, times, Zxx_l, Zxx_r

def plot_spectogram(times, frequencies, Zxx, segment=None, freq_limit=4000, log=False):
    plt.figure(figsize=(12, 6))
    Zxx_for_plot = np.abs(Zxx[:, segment[0]:segment[1]]) if segment else np.abs(Zxx)
    Zxx_for_plot = np.log10(Zxx_for_plot + 1e-6) if log else Zxx_for_plot
    times_for_plot = times[segment[0]:segment[1]] if segment else times
    plt.pcolormesh(times_for_plot, frequencies, Zxx_for_plot, shading='gouraud', cmap='Greys_r')
    plt.title('STFT Magnitude of Music Track')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.colorbar(label='Magnitude')
    plt.ylim([0, freq_limit])
    plt.show()

def get_window_data(Zxx, window_size=5):
    transposed_stft = Zxx.T
    zeros_for_window = np.zeros((window_size // 2, transposed_stft.shape[1]))
    windows = np.zeros((transposed_stft.shape[0], window_size, transposed_stft.shape[1]), dtype=np.float32)
    data_for_windows = np.concatenate([zeros_for_window, transposed_stft, zeros_for_window])
    
    for i in range(transposed_stft.shape[0]):
        windows[i] = data_for_windows[i:i+window_size]

    return windows


In [114]:
audio_files = [f'data/musdb/musdb18hq/train/{track}/mixture.wav' for track in os.listdir('data/musdb/musdb18hq/train')]
vocals_files = [f'data/musdb/musdb18hq/train/{track}/vocals.wav' for track in os.listdir('data/musdb/musdb18hq/train')]
audio_files.extend([f'data/ccmixter_corpus/{track}/mix.wav' for track in os.listdir('data/ccmixter_corpus')[:30]])
vocals_files.extend([f'data/ccmixter_corpus/{track}/source-02.wav' for track in os.listdir('data/ccmixter_corpus')[:30]])


def read_tf_file(file_path):
    file_path = file_path.numpy().decode("utf-8")
    sr, audio = wav.read(file_path)
    
    return audio.astype(np.float32)

def stft_transform(signal, sample_rate=44100, nperseg=2048, noverlap=2048 // 2):
    frequencies, times, Zxx_l = stft(signal[:, 0], fs=sample_rate, nperseg=nperseg, noverlap=noverlap, window='hamming')
    frequencies, times, Zxx_r = stft(signal[:, 1], fs=sample_rate, nperseg=nperseg, noverlap=noverlap, window='hamming')
    return Zxx_l, Zxx_r

def process_audio(audio_file, vocal_file):
    audio = read_tf_file(audio_file)
    Zxx_l, Zxx_r = stft_transform(audio)
    vocals_audio = read_tf_file(vocal_file)
    Zxx_l_v, Zxx_r_v = stft_transform(vocals_audio)
    return tf.abs(Zxx_l), tf.abs(Zxx_r), tf.abs(Zxx_l_v), tf.abs(Zxx_r_v)

def get_ideal_mask_tf(stft, stft_vocals):
    mask = get_ideal_binary_mask(stft_vocals, stft)
    return tf.convert_to_tensor(mask, dtype=tf.float32)

def get_window_data_tf(stft, window_size=9):
    transposed_stft = tf.transpose(stft)
    zeros_for_window = np.zeros((window_size // 2, transposed_stft.shape[1]))
    windows = np.zeros((transposed_stft.shape[0], window_size, transposed_stft.shape[1]), dtype=np.float32)
    data_for_windows = np.concatenate([zeros_for_window, transposed_stft, zeros_for_window])
    
    for i in range(transposed_stft.shape[0]):
        windows[i] = data_for_windows[i:i+window_size]

    return tf.convert_to_tensor(windows, dtype=tf.float32)


def process_with_tf(audio_file, vocal_file):
    stft_left, stft_right, stft_left_vocals, stft_right_vocals = tf.py_function(
        func=process_audio, inp=[audio_file, vocal_file], Tout=(tf.float32, tf.float32, tf.float32, tf.float32)
    )
    mask_left = tf.py_function(
        func=get_ideal_mask_tf, inp=[stft_left, stft_left_vocals], Tout=tf.float32
    )
    mask_right = tf.py_function(
        func=get_ideal_mask_tf, inp=[stft_right, stft_right_vocals], Tout=tf.float32
    )
    left_windows = tf.py_function(
        func=get_window_data_tf, inp=[stft_left], Tout=tf.float32
    )
    right_windows = tf.py_function(
        func=get_window_data_tf, inp=[stft_right], Tout=tf.float32
    )
    
    left_ds = tf.data.Dataset.from_tensor_slices(left_windows)
    right_ds = tf.data.Dataset.from_tensor_slices(right_windows)
    left_ds_vocals = tf.data.Dataset.from_tensor_slices(tf.transpose(mask_left))
    right_ds_vocals = tf.data.Dataset.from_tensor_slices(tf.transpose(mask_right))

    return tf.data.Dataset.zip(left_ds.concatenate(right_ds), left_ds_vocals.concatenate(right_ds_vocals))

In [123]:
dataset = tf.data.Dataset.from_tensor_slices((audio_files, vocals_files))
dataset = dataset.flat_map(process_with_tf)
dataset = dataset.map(lambda x, y: (tf.ensure_shape(x, (9, 1025)), tf.ensure_shape(y, (1025, ))) )
dataset = dataset.batch(32)
dataset = dataset.map(lambda x, y: (tf.ensure_shape(x, (32, 9, 1025)), tf.ensure_shape(y, (32, 1025))))
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [116]:
for batch in dataset.take(1):
    x_sample, y_sample = batch
    print("Sample input shape (X):", x_sample.shape)
    print("Sample output shape (Y):", y_sample.shape)

Sample input shape (X): (9, 1025)
Sample output shape (Y): (1025,)


In [105]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None))>

## Training model

In [36]:
for batch in dataset.take(1):
    x_sample, y_sample = batch
    print("Sample input shape (X):", x_sample.shape)
    print("Sample output shape (Y):", y_sample.shape)

Sample input shape (X): (32, 9, 1025)
Sample output shape (Y): (32, 1025)


In [10]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0']


In [26]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, 
    Conv2D, 
    MaxPooling2D, 
    UpSampling2D, 
    concatenate, 
    GlobalAveragePooling2D, 
    Dense, 
    Flatten, 
    Reshape, 
    ZeroPadding2D
)

In [74]:
freq_bins = 1025
window_size = 9

def create_model():
    # Input
    inputs = Input(shape=(window_size, freq_bins))
    inputs = Reshape((window_size, freq_bins, 1))(inputs)

    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Bottleneck
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)

    # Decoder
    up5 = UpSampling2D(size=(2, 2))(conv4)
    up5 = concatenate([up5, conv3])
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(up5)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)

    up6 = UpSampling2D(size=(2, 2))(conv5)
    up6 = concatenate([up6, conv2])
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    up7 = ZeroPadding2D(padding=((0, 1), (0, 1)))(up7)
    up7 = concatenate([up7, conv1])
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)

    # Output
    final_conv = Conv2D(1, (3, 3), activation='relu', padding='same')(conv7)
    pooled = GlobalAveragePooling2D()(final_conv)
    outputs = Dense(1025, activation='sigmoid')(pooled)

    model = Model(inputs=inputs, outputs=outputs, name="vocal_separator")
    return model

In [64]:
freq_bins = 1025
window_size = 9

def create_model2():
    inputs = tf.keras.Input(shape=(window_size, freq_bins), name="input_audio")
    inputs = tf.keras.layers.Reshape((9, 1025, 1))(inputs)
    
    x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    
    outputs = tf.keras.layers.Dense(freq_bins, activation="linear")(x)
    
    model = tf.keras.Model(inputs, outputs, name="vocal_separator")
    return model

In [117]:
freq_bins = 1025
window_size = 9

def create_model3():
    inputs = tf.keras.Input(shape=(window_size, freq_bins), name="input_audio")
    
    x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    
    outputs = tf.keras.layers.Dense(freq_bins, activation="sigmoid")(x)
    # outputs = tf.keras.layers.Reshape((freq_bins,))(outputs)
    
    model = tf.keras.Model(inputs, outputs, name="vocal_separator")
    return model

In [125]:
nn = create_model()

In [126]:
nn.summary()

Model: "vocal_separator"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ keras_tensor_237CL… │ (None, 9, 1025,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_95 (Conv2D)  │ (None, 9, 1025,   │        640 │ keras_tensor_237… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_96 (Conv2D)  │ (None, 9, 1025,   │     36,928 │ conv2d_95[1][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_25    │ (None, 4, 512,    │          0 │ conv2d_96[1][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_97 (Conv2D)  │ (None, 4, 512,    │     73,856 │ max_pooling2d_25… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_98 (Conv2D)  │ (None, 4, 512,    │    147,584 │ conv2d_97[1][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_26    │ (None, 2, 256,    │          0 │ conv2d_98[1][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_99 (Conv2D)  │ (None, 2, 256,    │    295,168 │ max_pooling2d_26… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_100 (Conv2D) │ (None, 2, 256,    │    590,080 │ conv2d_99[1][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_27    │ (None, 1, 128,    │          0 │ conv2d_100[1][0]  │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_101 (Conv2D) │ (None, 1, 128,    │  1,180,160 │ max_pooling2d_27… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_102 (Conv2D) │ (None, 1, 128,    │  2,359,808 │ conv2d_101[1][0]  │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_18    │ (None, 2, 256,    │          0 │ conv2d_102[1][0]  │
│ (UpSampling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_18      │ (None, 2, 256,    │          0 │ up_sampling2d_18… │
│ (Concatenate)       │ 768)              │            │ conv2d_100[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_103 (Conv2D) │ (None, 2, 256,    │  1,769,728 │ concatenate_18[1… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_104 (Conv2D) │ (None, 2, 256,    │    590,080 │ conv2d_103[1][0]  │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_19    │ (None, 4, 512,    │          0 │ conv2d_104[1][0]

 Total params: 7,784,323 (29.69 MB)

 Trainable params: 7,784,323 (29.69 MB)

 Non-trainable params: 0 (0.00 B)

In [119]:
nn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss="categorical_crossentropy",
    metrics=['accuracy', "mse"]
)
epochs = 1

In [121]:
history = nn.fit(dataset, epochs=epochs)

ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [127]:
for i in dataset.unbatch().take(5).batch(1):
    print(nn.predict(i[0]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 738ms/step
[[0.49917254 0.49966943 0.50107074 ... 0.50093186 0.50136745 0.50158906]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
[[0.49873325 0.49949393 0.5016391  ... 0.5014266  0.50209343 0.5024327 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
[[0.49821535 0.499287   0.5023093  ... 0.50200987 0.50294936 0.5034272 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
[[0.49761778 0.49904826 0.5030825  ... 0.5026828  0.5039369  0.5045748 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
[[0.49702236 0.49881035 0.503853   ... 0.50335336 0.50492084 0.5057181 ]]


In [10]:
FRAME_LENGTH = 2048
HOP_LENGTH = 1024
SR = 44100
NUM_FRAMES = 25
TARGET_FRAME_INDEX = NUM_FRAMES // 2
FREQ_BINS = FRAME_LENGTH // 2 + 1

In [40]:
from tensorflow.keras.saving import load_model

def extract_central_frame(x):
    return x[:, TARGET_FRAME_INDEX, :]

model_path = 'models/best_rnn_model.keras'
model = load_model(model_path, custom_objects={"extract_central_frame": extract_central_frame})
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1025, 25)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ permute_3 (Permute)             │ (None, 25, 1025)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 25, 256)        │     1,312,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 25, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 25, 256)        │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 25, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 25, 1025)       │       263,425 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_3 (Lambda)               │ (None, 1025)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,304,517 (24.05 MB)

 Trainable params: 2,101,505 (8.02 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,203,012 (16.03 MB)

In [41]:
import os

mix_files_test = [f'data/musdb/musdb18hq/test/{track}/mixture.wav' for track in os.listdir('data/musdb/musdb18hq/test')][:1]
vocal_files_test = [f'data/musdb/musdb18hq/test/{track}/vocals.wav' for track in os.listdir('data/musdb/musdb18hq/test')][:1]

In [42]:
file_dataset_test = tf.data.Dataset.from_tensor_slices((mix_files_test, vocal_files_test))

In [43]:
file_dataset_test

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>

In [44]:
FRAME_LENGTH = 2048
HOP_LENGTH = 1024
SR = 44100
NUM_FRAMES = 25
TARGET_FRAME_INDEX = NUM_FRAMES // 2
FREQ_BINS = FRAME_LENGTH // 2 + 1

from scipy.io import wavfile

def load_audio_with_scipy(file_path, target_sr=SR, frame_length=10 * SR):
    file_path = file_path.numpy().decode("utf-8")
    sr, audio = wavfile.read(file_path)
    if audio.dtype == np.int16:
        audio = audio / 32768.0
    if len(audio.shape) > 1:
        audio = np.mean(audio, axis=1)
    if sr != target_sr:
        from scipy.signal import resample
        audio = resample(audio, int(len(audio) * target_sr / sr))
    return audio

def compute_spectrogram(audio, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH):
    _, _, Zxx = stft(audio, fs=SR, nperseg=frame_length, noverlap=frame_length - hop_length)
    return np.abs(Zxx)

def create_binary_mask(vocal_spec, mix_spec, threshold=1.05):
    return (vocal_spec > threshold * mix_spec).astype(np.int32)

def get_window_from_spectrogram(spectrogram, start_idx, num_frames=NUM_FRAMES):
    num_freq_bins, num_time_frames = spectrogram.shape
    window = np.zeros((num_freq_bins, num_frames))

    start = max(0, start_idx)
    end = min(start_idx + num_frames, num_time_frames)

    insert_start = max(0, -start_idx)
    insert_end = insert_start + (end - start)

    window[:, insert_start:insert_end] = spectrogram[:, start:end]

    return window

def preprocess_pair_with_scipy(mix_path, vocal_path):
    mix_audio = load_audio_with_scipy(mix_path)
    vocal_audio = load_audio_with_scipy(vocal_path)

    mix_spec = compute_spectrogram(mix_audio)
    vocal_spec = compute_spectrogram(vocal_audio)
    mask = create_binary_mask(vocal_spec, mix_spec)

    X, y = [], []
    for i in range(-(NUM_FRAMES // 2), mix_spec.shape[1] - (NUM_FRAMES // 2)):
        window = get_window_from_spectrogram(mix_spec, i, NUM_FRAMES)
        target_frame = mask[:, max(0, min(i + TARGET_FRAME_INDEX, mask.shape[1] - 1))]
        X.append(window)
        y.append(target_frame)
    return np.array(X), np.array(y)

def tf_preprocess(mix_path, vocal_path):
    X, y = tf.py_function(
        func=preprocess_pair_with_scipy,
        inp=[mix_path, vocal_path],
        Tout=[tf.float32, tf.float32],
    )
    X.set_shape([None, FREQ_BINS, NUM_FRAMES])
    y.set_shape([None, FREQ_BINS])
    return X, y

In [45]:
test_dataset = file_dataset_test.map(tf_preprocess)

In [46]:
BATCH_SIZE = 4

In [47]:
test_dataset = test_dataset.unbatch().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [48]:
pred = model.predict(test_dataset)

2160/2160 ━━━━━━━━━━━━━━━━━━━━ 48s 19ms/step


In [28]:
mix_files_test[0]

'data/musdb/musdb18hq/test/Al James - Schoolboy Facination/mixture.wav'

In [56]:
(pred > 0.5).astype(np.int32).T

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [50]:
from vocalseparatordiploma.preprocessing import *

In [51]:
track = read_track(mix_files_test[0])

In [52]:
track

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)

In [53]:
left = compute_stft(track[:, 0])
right = compute_stft(track[:, 1])

In [54]:
left.shape

(1025, 8637)

In [57]:
windows_left = generate_windows(left)

In [59]:
windows_left

<generator object generate_windows at 0x0000026E77D7CBF0>

In [58]:
model.predict(windows_left)

ValueError: When passing a Python generator to a Keras model, the generator must return a tuple, either (input,) or (inputs, targets) or (inputs, targets, sample_weights). Received: [[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]

In [60]:
def predict_with_generator(model, spectrogram, window_size=25, batch_size=32):
    """
    Выполняет предсказание масок для спектрограммы с использованием генератора окон.

    Args:
        model (tf.keras.Model): Обученная модель.
        spectrogram (np.ndarray): Спектрограмма размером (freq_bins, time_steps).
        window_size (int): Размер окна для скользящих окон.
        batch_size (int): Размер батча для предсказания.

    Returns:
        np.ndarray: Предсказанная маска размером (freq_bins, time_steps).
    """
    spectrum_width, length = spectrogram.shape
    predicted_mask = np.zeros((spectrum_width, length))

    # Генератор для подготовки данных
    def data_generator():
        windows = []
        indices = []
        zeros_for_window = np.zeros((spectrum_width, window_size // 2))
        data_for_windows = np.concatenate([zeros_for_window, spectrogram, zeros_for_window], axis=1)

        for i in range(length):
            window = data_for_windows[:, i:i+window_size]
            windows.append(np.expand_dims(window, axis=-1))
            indices.append(i)

            if len(windows) == batch_size:  # Если набрался батч
                yield np.array(windows), indices
                windows = []
                indices = []

        # Последний неполный батч
        if windows:
            yield np.array(windows), indices

    # Выполняем предсказание
    for batch_data, batch_indices in data_generator():
        batch_predictions = model.predict(batch_data, verbose=0)
        for idx, pred_mask in zip(batch_indices, batch_predictions):
            predicted_mask[:, idx] = pred_mask

    return predicted_mask

In [63]:
mask_l = predict_with_generator(model, np.abs(left))

In [66]:
(mask_l > 0.5)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])